In [1]:
import os
import glob
from datetime import datetime
import basic_plots_from_netcdf

from utility_programs.utils import get_var_names, str_to_ut
import numpy as np
import xarray as xr
from cartopy import crs as ccrs
import matplotlib.pyplot as plt

# Settings

In [2]:
top_path = '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/'

In [3]:
out_dir = '/glade/u/home/abukowski/plots/'

In [4]:
columns = ['edens', 'Rho']

In [5]:
do_map = True

In [6]:
time_lims_set = [201111211030, 2011112120]

In [7]:
alts = [450, 840]

In [8]:
types = ['bandpass',['alt_int', 'bandpass']]

# Find data paths

In [9]:
dirs_with_data = []

for root, dirs, files in os.walk(top_path, 
                                topdown=True):
    if len(files) > 1:
        dirs_with_data.append(root)

In [10]:
dirs_with_data

['/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/Full',
 '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/rotB/bzfull',
 '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/rotB/bz15/byneg',
 '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/rotB/bz15/bypos',
 '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/rotB/bz8/byneg',
 '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/rotB/bz8/bypos',
 '/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/Half']

# Run:

- loop thru the options, set up a command, run the program

In [11]:
import importlib

In [12]:
basic_plots_from_netcdf = importlib.reload(basic_plots_from_netcdf)

In [30]:
for path in dirs_with_data:
    for col in columns:
        
        time_lims = [None, None]
        if col =='Rho':
            file_list = glob.glob(os.path.join(path, 'GITM' + '*.nc'))
        elif col =='edens':
            file_list = glob.glob(os.path.join(path, 'SAMI_REGRID' + '*.nc'))
            
        if len(file_list) == 0:
            raise ValueError
        file_list = np.sort(file_list)
        
        dtime_lim_0 = str_to_ut(str(int(time_lims_set[0])))
        dtime_lim_1 = str_to_ut(str(int(time_lims_set[1])))
        time_list = []
        for fname in file_list:
            t_str = fname.split('_')[-1].split('.')[0]
            time_list.append(datetime.strptime(t_str, '%Y-%m-%dT%H-%M-%S'))
        time_list = np.array(time_list)
        time_lims[0] = np.argmin(np.abs(time_list - dtime_lim_0))
        time_lims[1] = np.argmin(np.abs(time_list - dtime_lim_1))

        file_list = file_list[time_lims[0]:time_lims[1]]
        

        ds0 = xr.open_dataset(file_list[0])
        drops = []
        for v in ds0.data_vars:
            if v not in col:
                drops.append(v)
        del ds0  # save memory
        
        print('Reading in {} files...'.format(len(file_list)))
        ds = [xr.open_dataset(f, drop_variables=drops) for f in file_list]
        ds = xr.concat(ds, dim='time')
        print('Done reading files.')
        
        if col =='edens':
            
            tec = basic_plots_from_netcdf.run_processing_options(ds.copy(), ['alt_int','bandpass'])
        
        ds = basic_plots_from_netcdf.run_processing_options(ds, 'bandpass')
        
        
        # make output dir
        out_path = os.path.join(out_dir, path.replace(top_path, ''), col)
        

        
        if col =='edens':
            tec_out_path = os.path.join(out_dir, path.replace(top_path, ''), 'tec')
            # os.makedirs(tec_out_path)
        for t in range(ds[col].shape[0]):
            
            for a in alts:
            
                p = ds[col].isel(time=t).sel(
                        alt=a, method='nearest').plot(
                    transform=ccrs.PlateCarree(),
                    subplot_kws={"projection": ccrs.PlateCarree()},
                    x='lon', y='lat',vmin=-3, vmax=3, cmap='bwr',
                    size=6,          )
                p.axes.coastlines()
                p.axes.gridlines(alpha=0.6)
                try:
                    plt.savefig(os.path.join(out_path,str(a), str(t).rjust(3,'0')))
                except FileNotFoundError:
                    os.makedirs(os.path.join(out_path,str(a)))
                    plt.savefig(os.path.join(out_path,str(a), str(t).rjust(3,'0')))
                plt.close()

            if col =='edens':
                p = tec[col].isel(time=t).plot(
                    transform=ccrs.PlateCarree(),
                    subplot_kws={"projection": ccrs.PlateCarree()},
                    x='lon', y='lat',vmin=-3, vmax=3, cmap='bwr',
                    size=6)
                p.axes.coastlines()
                p.axes.gridlines(alpha=0.6)
                try:
                    plt.savefig(os.path.join(tec_out_path, str(t).rjust(3,'0')))
                except FileNotFoundError:
                    os.makedirs(tec_out_path)
                    plt.savefig(os.path.join(tec_out_path, str(t).rjust(3,'0')))
                plt.close()
        
        del ds, tec


Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.
Reading in 114 files...
Done reading files.


In [28]:
ds

<xarray.Dataset>
Dimensions:  (time: 114, alt: 40, lat: 180, lon: 90)
Coordinates:
  * time     (time) datetime64[ns] 2011-11-21T10:30:00 ... 2011-11-21T19:55:00
  * alt      (alt) float64 200.0 250.0 300.0 350.0 ... 2.05e+03 2.1e+03 2.15e+03
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 86.0 87.0 88.0 89.0
  * lon      (lon) float64 0.0 4.0 8.0 12.0 16.0 ... 344.0 348.0 352.0 356.0
Data variables:
    edens    (time, lat, lon, alt) float64 0.0001488 3.358e-05 ... 0.0001312

In [29]:
tec

<xarray.Dataset>
Dimensions:  (time: 114, lat: 180, lon: 90)
Coordinates:
  * time     (time) datetime64[ns] 2011-11-21T10:30:00 ... 2011-11-21T19:55:00
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 86.0 87.0 88.0 89.0
  * lon      (lon) float64 0.0 4.0 8.0 12.0 16.0 ... 344.0 348.0 352.0 356.0
Data variables:
    edens    (time, lat, lon) float64 0.0002613 0.0002613 ... 0.000119 0.000119

In [ ]:
# for plot_dir in dirs_with_data:
#     for alt in alts:
#         for proc_opt in types:
#             for col in columns:
            
#                 out_path = os.path.join(out_dir, col+'-'+proc_opt[0],
#                                         plot_dir.replace(top_path, ''))
                                        
#                 if col == 'edens':
#                     basic_plots_from_netcdf.autoplot(data_dir=plot_dir,
#                              columns_to_plot=col,
#                              model='SAMI_REGRID',
#                              output_dir=out_path,
#                              show_map=True,
#                              time_lims=time_lims,
#                              cut_dict={'alt':alt},
#                              process_options=proc_opt,
#                              plot_arg_dict={'size':6,
#                                            'vmin':-3,
#                                            'vmax':3,
#                                            'cmap':'bwr'})
                
#                 elif col =='Rho' and proc_opt != 'alt_int':

#                     basic_plots_from_netcdf.autoplot(data_dir=plot_dir,
#                              columns_to_plot=col,
#                              model='GITM',
#                              output_dir=out_path,
#                              show_map=True,
#                              time_lims=time_lims,
#                              cut_dict={'alt':alt},
#                              process_options=proc_opt,
#                              plot_arg_dict={'size':6,
#                                            'vmin':-3,
#                                            'vmax':3,
#                                            'cmap':'bwr'})